In [3]:
from Utils import Connect2Mongo
import pandas as pd
import QuantLib as ql
from datetime import datetime, timedelta
from math import ceil
import os
option_data_path = "I:\\GitGroups\\HomeLab\\data-collection\\TXO"

In [10]:
def GetExpireDate(symbol:str, ttm:str) -> ql.Date:
    nthWeekMapping = {
        "O":3,
        "F":3,
        "A":3
    }
    nWeek = nthWeekMapping.get(symbol[2], symbol[2])
    yStr, mStr = ttm.split('/')
    return ql.Date.nthWeekday(int(nWeek), ql.Wednesday, int(mStr), int(yStr)).to_date().strftime("%Y-%m-%d")

def week_of_month(dt):
    """ Returns the week of the month for the specified date.
    """

    first_day = dt.replace(day=1)

    dom = dt.day
    adjusted_dom = dom + first_day.weekday()

    return int(ceil(adjusted_dom/7.0))

def get_expire_date(dtStr:str) -> ql.Date:
    dt = datetime.strptime(dtStr, "%Y-%m-%d")
    yStr, mStr, dStr = dtStr.split('-')
    for week in range(1, 6):
        try:
            # expire at nth wednesday
            expire_date = ql.Date.nthWeekday(week, ql.Wednesday, int(mStr), int(yStr)).to_date()
            if expire_date > dt.date():
                break
        except:
            # expire at next month first wednesday
            expire_date += timedelta(days=7)
    if dt.date() > expire_date:
        expire_date += timedelta(days=7)
    return expire_date.strftime("%Y-%m-%d")

def get_maturity(dtStr):
    y, m, d = dtStr.split('-')
    dt = datetime.strptime(dtStr, "%Y-%m-%d")
    w_of_month = week_of_month(dt)
    ttm = y + m
    if w_of_month != 3:
        ttm += "W" + str(w_of_month)
    return ttm

In [4]:
table = Connect2Mongo("TWSE.HistoricalPrice.Index.Intraday")

In [5]:
full_df = list(table.find({"Date":{"$lte":"2024-01-31", "$gte":"2023-01-01"}, "IndexName":"發行量加權股價指數"}))

full_df = pd.DataFrame(full_df)
full_df['DateTime'] = full_df['Date'] + " " + full_df['Time']
del full_df['_id']

In [6]:
grouped = full_df.groupby("Date")
dfs = {}
for group in grouped.groups:
    dfs[group] = grouped.get_group(group)

In [7]:
middles = {}
for k, v in dfs.items():
    tmp_middle = v[v.Time.isin(["09:00:00", "09:10:00", "09:20:00", "09:30:00"])].Close.mean()
    selected_middle = int(round(tmp_middle / 100) * 100)
    middles[k] = {"middle":selected_middle}


In [ ]:
for k, v in middles.items():
    try:
        v['expire_date'] = get_expire_date(k)
        if v['expire_date'] not in full_df.Date.unique():
            for i in range(1, 20):
                tmp_dtStr = (datetime.strptime(v['expire_date'], "%Y-%m-%d") + timedelta(days=i)).strftime("%Y-%m-%d")
                if tmp_dtStr in full_df.Date.to_list():
                    v['expire_date'] = tmp_dtStr
        v['expire_price'] = full_df[full_df.Date==v['expire_date']].sort_values("Time").Close.iloc[-1]
        v['expire_diff'] = round(abs(v['expire_price'] - v['middle']), 2)
        period_price = full_df[(full_df.DateTime.between(k + " 09:30:00", v['expire_date'] + " 13:31:00"))].Close
        v['max_price_before_expire'] = period_price.max()
        v['min_price_before_expire'] = period_price.min()
        v['expire_diff_max'] = round(abs(v['max_price_before_expire'] - v['middle']), 2)
        v['expire_diff_min'] = round(abs(v['min_price_before_expire'] - v['middle']), 2)
        v['used_strike'] = {
            'buy_call': v['middle'] + 100,
            'buy_put': v['middle'] - 100,
            'sell_call': v['middle'] + 300,
            'sell_put': v['middle'] - 300,
        }
        v['Maturity'] = get_maturity(v['expire_date'])
        v['days_to_expire'] = (datetime.strptime(v['expire_date'], "%Y-%m-%d") - datetime.strptime(k, "%Y-%m-%d")).days
    except:
        print(v['expire_date'])


In [20]:
wins = []
losses = []
for k, v in middles.items():
    if v['expire_diff_max'] >= 207:
        wins.append(k)
    else:
        losses.append(k)

In [23]:
num_win = len(wins)
num_loss = len(losses)
win_rate = round(num_win / (num_win + num_loss) * 100, 2)

num_win, num_loss, win_rate

(92, 168, 35.38)

In [111]:
def read_intraday_data(date, strikes, maturity):
    intraday_data = pd.read_csv(os.path.join(option_data_path, f"TXO_{date.replace('-', '_')}.csv"), encoding='utf-8', index_col=0)
    intraday_data = intraday_data[(intraday_data.tradingtime.between(84500, 134500))]
    return intraday_data[(intraday_data.maturity==maturity) & (intraday_data.exprice.isin(strikes))]

In [169]:
def daytrade_pnl(entry_date, v):
    
    strikes = list(v['used_strike'].values())
    maturity = v['Maturity']

    used_data = read_intraday_data(entry_date, strikes, maturity)

    buy_call_data = used_data[(used_data['C/P'] == "C") & (used_data.exprice==v['used_strike']['buy_call']) & (used_data.tradingtime <= 93000)]
    sell_call_data = used_data[(used_data['C/P'] == "C") & (used_data.exprice==v['used_strike']['sell_call']) & (used_data.tradingtime <= 93000)]
    buy_put_data = used_data[(used_data['C/P'] == "P") & (used_data.exprice==v['used_strike']['buy_put']) & (used_data.tradingtime <= 93000)]
    sell_put_data = used_data[(used_data['C/P'] == "P") & (used_data.exprice==v['used_strike']['sell_put']) & (used_data.tradingtime <= 93000)]
    
    buy_call_exit_data = used_data[(used_data['C/P'] == "C") & (used_data.exprice==v['used_strike']['buy_call']) & (used_data.tradingtime > 93000)]
    sell_call_exit_data = used_data[(used_data['C/P'] == "C") & (used_data.exprice==v['used_strike']['sell_call']) & (used_data.tradingtime > 93000)]
    buy_put_exit_data = used_data[(used_data['C/P'] == "P") & (used_data.exprice==v['used_strike']['buy_put']) & (used_data.tradingtime > 93000)]
    sell_put_exit_data = used_data[(used_data['C/P'] == "P") & (used_data.exprice==v['used_strike']['sell_put']) & (used_data.tradingtime > 93000)]

    buy_call_entry = buy_call_data.iloc[-1].price
    sell_call_entry = sell_call_data.iloc[-1].price
    buy_put_entry = buy_put_data.iloc[-1].price
    sell_put_entry = sell_put_data.iloc[-1].price

    buy_call_exit = buy_call_exit_data.iloc[-1].price
    sell_call_exit = sell_call_exit_data.iloc[-1].price
    buy_put_exit= buy_put_exit_data.iloc[-1].price
    sell_put_exit = sell_put_exit_data.iloc[-1].price

    buy_call_pnl = buy_call_exit - buy_call_entry
    sell_call_pnl = sell_call_entry - sell_call_exit
    buy_put_pnl = buy_put_exit - buy_put_entry
    sell_put_pnl = sell_put_entry - sell_put_exit

    pnl = round(buy_call_pnl + sell_call_pnl + buy_put_pnl + sell_put_pnl, 2)
    entry_cost = round(-buy_call_entry + sell_call_entry - buy_put_entry + sell_put_entry, 2)
    minimum_diff_index = int(abs(entry_cost)) + 101
    
    v['daytrade_pnl'] = pnl
    v['entry_cost'] = entry_cost
    
    print(f"==================== daytrade report of {entry_date}====================")
    print(f"middle line : {v['middle']}")
    print(f"used strike : {v['used_strike']}")
    print(f"entry of each contract : buy_call[{buy_call_entry}], sell_call[{sell_call_entry}], buy_put[{buy_put_entry}], sell_put[{sell_put_entry}]")
    print(f"entry cost : {entry_cost}")
    print(f"minimum diff by entry : {minimum_diff_index}")
    print(f"max_diff_between_entry_and_expire : {v['expire_diff_max']}")
    print(f"exit of each contract : buy_call[{buy_call_exit}], sell_call[{sell_call_exit}], buy_put[{buy_put_exit}], sell_put[{sell_put_exit}]")
    print(f"pnl : {pnl}")
    return pnl

In [170]:
def expire_pnl(entry_date, v):
    
    strikes = list(v['used_strike'].values())
    maturity = v['Maturity']
    
    entry_date_data = read_intraday_data(entry_date, strikes, maturity)
    exit_date_data = read_intraday_data(v['expire_date'], strikes, maturity)

    buy_call_data = entry_date_data[(entry_date_data['C/P'] == "C") & (entry_date_data.exprice==v['used_strike']['buy_call']) & (entry_date_data.tradingtime <= 93000)]
    sell_call_data = entry_date_data[(entry_date_data['C/P'] == "C") & (entry_date_data.exprice==v['used_strike']['sell_call']) & (entry_date_data.tradingtime <= 93000)]
    buy_put_data = entry_date_data[(entry_date_data['C/P'] == "P") & (entry_date_data.exprice==v['used_strike']['buy_put']) & (entry_date_data.tradingtime <= 93000)]
    sell_put_data = entry_date_data[(entry_date_data['C/P'] == "P") & (entry_date_data.exprice==v['used_strike']['sell_put']) & (entry_date_data.tradingtime <= 93000)]
    
    buy_call_exit_data = exit_date_data[(exit_date_data['C/P'] == "C") & (exit_date_data.exprice==v['used_strike']['buy_call'])]
    sell_call_exit_data = exit_date_data[(exit_date_data['C/P'] == "C") & (exit_date_data.exprice==v['used_strike']['sell_call'])]
    buy_put_exit_data = exit_date_data[(exit_date_data['C/P'] == "P") & (exit_date_data.exprice==v['used_strike']['buy_put'])]
    sell_put_exit_data = exit_date_data[(exit_date_data['C/P'] == "P") & (exit_date_data.exprice==v['used_strike']['sell_put'])]

    buy_call_entry = buy_call_data.iloc[-1].price
    sell_call_entry = sell_call_data.iloc[-1].price
    buy_put_entry = buy_put_data.iloc[-1].price
    sell_put_entry = sell_put_data.iloc[-1].price

    buy_call_exit = buy_call_exit_data.iloc[-1].price
    sell_call_exit = sell_call_exit_data.iloc[-1].price
    buy_put_exit = buy_put_exit_data.iloc[-1].price
    sell_put_exit = sell_put_exit_data.iloc[-1].price

    buy_call_pnl = buy_call_exit - buy_call_entry
    sell_call_pnl = sell_call_entry - sell_call_exit
    buy_put_pnl = buy_put_exit - buy_put_entry
    sell_put_pnl = sell_put_entry - sell_put_exit

    pnl = round(buy_call_pnl + sell_call_pnl + buy_put_pnl + sell_put_pnl, 2)
    entry_cost = round(-buy_call_entry + sell_call_entry - buy_put_entry + sell_put_entry, 2)
    minimum_diff_index = int(abs(entry_cost)) + 101
    
    v['expire_pnl'] = pnl
    v['entry_cost_to_expire'] = entry_cost
    
    print(f"==================== expire report of {entry_date}====================")
    print(f"middle line : {v['middle']}")
    print(f"used strike : {v['used_strike']}")
    print(f"days to expire : {v['days_to_expire']}")
    print(f"entry of each contract : buy_call[{buy_call_entry}], sell_call[{sell_call_entry}], buy_put[{buy_put_entry}], sell_put[{sell_put_entry}]")
    print(f"entry cost : {entry_cost}")
    print(f"minimum diff by entry : {minimum_diff_index}")
    print(f"max_diff_between_entry_and_expire : {v['expire_diff_max']}")
    print(f"exit of each contract : buy_call[{buy_call_exit}], sell_call[{sell_call_exit}], buy_put[{buy_put_exit}], sell_put[{sell_put_exit}]")
    print(f"pnl : {pnl}")
    return pnl

In [171]:
for k, v in middles.items():
    try:
        daytrade_pnl(k, v)
    except:
        pass

==================== daytrade report of 2023-10-05====================
middle line : 16400
used strike : {'buy_call': 16500, 'buy_put': 16300, 'sell_call': 16700, 'sell_put': 16100}
entry of each contract : buy_call[102.0], sell_call[45.0], buy_put[149.0], sell_put[79.0]
entry cost : -127.0
minimum diff by entry : 228
max_diff_between_entry_and_expire : 329.5
exit of each contract : buy_call[140.0], sell_call[63.0], buy_put[100.0], sell_put[52.0]
pnl : -2.0
==================== daytrade report of 2023-10-06====================
middle line : 16500
used strike : {'buy_call': 16600, 'buy_put': 16400, 'sell_call': 16800, 'sell_put': 16200}
entry of each contract : buy_call[106.0], sell_call[44.5], buy_put[116.0], sell_put[59.0]
entry cost : -118.5
minimum diff by entry : 219
max_diff_between_entry_and_expire : 229.5
exit of each contract : buy_call[112.0], sell_call[45.0], buy_put[100.0], sell_put[50.0]
pnl : -1.5
==================== daytrade report of 2023-10-11====================
middl

c:\users\kevin\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3343: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


==================== daytrade report of 2023-10-25====================
middle line : 16400
used strike : {'buy_call': 16500, 'buy_put': 16300, 'sell_call': 16700, 'sell_put': 16100}
entry of each contract : buy_call[82.0], sell_call[25.0], buy_put[94.0], sell_put[42.0]
entry cost : -109.0
minimum diff by entry : 210
max_diff_between_entry_and_expire : 59.34
exit of each contract : buy_call[61.0], sell_call[14.5], buy_put[106.0], sell_put[46.5]
pnl : -3.0
==================== daytrade report of 2023-10-26====================
middle line : 16200
used strike : {'buy_call': 16300, 'buy_put': 16100, 'sell_call': 16500, 'sell_put': 15900}
entry of each contract : buy_call[53.0], sell_call[13.5], buy_put[140.0], sell_put[69.0]
entry cost : -110.5
minimum diff by entry : 211
max_diff_between_entry_and_expire : 15.35
exit of each contract : buy_call[40.0], sell_call[8.9], buy_put[153.0], sell_put[74.0]
pnl : -0.4
==================== daytrade report of 2023-10-27====================
middle line

==================== daytrade report of 2023-11-20====================
middle line : 17200
used strike : {'buy_call': 17300, 'buy_put': 17100, 'sell_call': 17500, 'sell_put': 16900}
entry of each contract : buy_call[47.0], sell_call[6.8], buy_put[33.5], sell_put[5.9]
entry cost : -67.8
minimum diff by entry : 168
max_diff_between_entry_and_expire : 221.17
exit of each contract : buy_call[35.0], sell_call[4.7], buy_put[37.5], sell_put[5.9]
pnl : -5.9
==================== daytrade report of 2023-11-21====================
middle line : 17300
used strike : {'buy_call': 17400, 'buy_put': 17200, 'sell_call': 17600, 'sell_put': 17000}
entry of each contract : buy_call[49.5], sell_call[4.6], buy_put[13.0], sell_put[2.4]
entry cost : -55.5
minimum diff by entry : 156
max_diff_between_entry_and_expire : 121.17
exit of each contract : buy_call[58.0], sell_call[4.4], buy_put[4.0], sell_put[0.8]
pnl : 1.3
==================== daytrade report of 2023-11-22====================
middle line : 17300
use

==================== daytrade report of 2023-12-14====================
middle line : 17600
used strike : {'buy_call': 17700, 'buy_put': 17500, 'sell_call': 17900, 'sell_put': 17300}
entry of each contract : buy_call[108.0], sell_call[42.0], buy_put[75.0], sell_put[29.0]
entry cost : -112.0
minimum diff by entry : 213
max_diff_between_entry_and_expire : 143.54
exit of each contract : buy_call[107.0], sell_call[38.0], buy_put[70.0], sell_put[27.0]
pnl : 0.0
==================== daytrade report of 2023-12-15====================
middle line : 17700
used strike : {'buy_call': 17800, 'buy_put': 17600, 'sell_call': 18000, 'sell_put': 17400}
entry of each contract : buy_call[85.0], sell_call[28.0], buy_put[76.0], sell_put[31.5]
entry cost : -101.5
minimum diff by entry : 202
max_diff_between_entry_and_expire : 38.82
exit of each contract : buy_call[66.0], sell_call[20.5], buy_put[91.0], sell_put[34.5]
pnl : 0.5
==================== daytrade report of 2023-12-18====================
middle line 

==================== daytrade report of 2024-01-17====================
middle line : 17300
used strike : {'buy_call': 17400, 'buy_put': 17200, 'sell_call': 17600, 'sell_put': 17000}
entry of each contract : buy_call[100.0], sell_call[33.5], buy_put[82.0], sell_put[35.0]
entry cost : -113.5
minimum diff by entry : 214
max_diff_between_entry_and_expire : 644.98
exit of each contract : buy_call[61.0], sell_call[18.0], buy_put[123.0], sell_put[60.0]
pnl : -7.5
==================== daytrade report of 2024-01-18====================
middle line : 17200
used strike : {'buy_call': 17300, 'buy_put': 17100, 'sell_call': 17500, 'sell_put': 16900}
entry of each contract : buy_call[99.0], sell_call[31.5], buy_put[62.0], sell_put[23.0]
entry cost : -106.5
minimum diff by entry : 207
max_diff_between_entry_and_expire : 744.98
exit of each contract : buy_call[97.0], sell_call[31.0], buy_put[58.0], sell_put[20.0]
pnl : -2.5
==================== daytrade report of 2024-01-19====================
middle li

In [172]:
for k, v in middles.items():
    try:
        expire_pnl(k, v)
    except:
        pass

==================== expire report of 2023-10-05====================
middle line : 16400
used strike : {'buy_call': 16500, 'buy_put': 16300, 'sell_call': 16700, 'sell_put': 16100}
days to expire : 6
entry of each contract : buy_call[102.0], sell_call[45.0], buy_put[149.0], sell_put[79.0]
entry cost : -127.0
minimum diff by entry : 228
max_diff_between_entry_and_expire : 329.5
exit of each contract : buy_call[260.0], sell_call[125.0], buy_put[17.5], sell_put[7.2]
pnl : 18.3
==================== expire report of 2023-10-06====================
middle line : 16500
used strike : {'buy_call': 16600, 'buy_put': 16400, 'sell_call': 16800, 'sell_put': 16200}
days to expire : 5
entry of each contract : buy_call[106.0], sell_call[44.5], buy_put[116.0], sell_put[59.0]
entry cost : -118.5
minimum diff by entry : 219
max_diff_between_entry_and_expire : 229.5
exit of each contract : buy_call[186.0], sell_call[76.0], buy_put[28.0], sell_put[10.5]
pnl : 9.0


c:\users\kevin\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3343: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


==================== expire report of 2023-10-11====================
middle line : 16700
used strike : {'buy_call': 16800, 'buy_put': 16600, 'sell_call': 17000, 'sell_put': 16400}
days to expire : 7
entry of each contract : buy_call[150.0], sell_call[66.0], buy_put[119.0], sell_put[65.0]
entry cost : -138.0
minimum diff by entry : 239
max_diff_between_entry_and_expire : 125.91
exit of each contract : buy_call[22.5], sell_call[5.1], buy_put[212.0], sell_put[108.0]
pnl : -16.6
==================== expire report of 2023-10-12====================
middle line : 16700
used strike : {'buy_call': 16800, 'buy_put': 16600, 'sell_call': 17000, 'sell_put': 16400}
days to expire : 6
entry of each contract : buy_call[148.0], sell_call[69.0], buy_put[95.0], sell_put[48.0]
entry cost : -126.0
minimum diff by entry : 227
max_diff_between_entry_and_expire : 125.91
exit of each contract : buy_call[22.5], sell_call[5.1], buy_put[212.0], sell_put[108.0]
pnl : -4.6
==================== expire report of 2023

==================== expire report of 2023-11-13====================
middle line : 16800
used strike : {'buy_call': 16900, 'buy_put': 16700, 'sell_call': 17100, 'sell_put': 16500}
days to expire : 2
entry of each contract : buy_call[75.0], sell_call[15.0], buy_put[14.0], sell_put[2.6]
entry cost : -71.4
minimum diff by entry : 172
max_diff_between_entry_and_expire : 343.88
exit of each contract : buy_call[223.0], sell_call[23.0], buy_put[0.3], sell_put[0.3]
pnl : 128.6
==================== expire report of 2023-11-14====================
middle line : 16900
used strike : {'buy_call': 17000, 'buy_put': 16800, 'sell_call': 17200, 'sell_put': 16600}
days to expire : 1
entry of each contract : buy_call[24.0], sell_call[1.5], buy_put[14.5], sell_put[2.0]
entry cost : -35.0
minimum diff by entry : 136
max_diff_between_entry_and_expire : 243.88
exit of each contract : buy_call[123.0], sell_call[0.1], buy_put[0.1], sell_put[0.1]
pnl : 87.9
==================== expire report of 2023-11-15=======

==================== expire report of 2023-12-07====================
middle line : 17400
used strike : {'buy_call': 17500, 'buy_put': 17300, 'sell_call': 17700, 'sell_put': 17100}
days to expire : 6
entry of each contract : buy_call[74.0], sell_call[29.5], buy_put[140.0], sell_put[66.0]
entry cost : -118.5
minimum diff by entry : 219
max_diff_between_entry_and_expire : 128.72
exit of each contract : buy_call[81.0], sell_call[22.5], buy_put[44.0], sell_put[12.0]
pnl : -28.0
==================== expire report of 2023-12-08====================
middle line : 17400
used strike : {'buy_call': 17500, 'buy_put': 17300, 'sell_call': 17700, 'sell_put': 17100}
days to expire : 5
entry of each contract : buy_call[112.0], sell_call[47.0], buy_put[89.0], sell_put[39.5]
entry cost : -114.5
minimum diff by entry : 215
max_diff_between_entry_and_expire : 128.72
exit of each contract : buy_call[81.0], sell_call[22.5], buy_put[44.0], sell_put[12.0]
pnl : -24.0
==================== expire report of 2023-1

==================== expire report of 2024-01-10====================
middle line : 17500
used strike : {'buy_call': 17600, 'buy_put': 17400, 'sell_call': 17800, 'sell_put': 17200}
days to expire : 7
entry of each contract : buy_call[106.0], sell_call[46.0], buy_put[117.0], sell_put[60.0]
entry cost : -117.0
minimum diff by entry : 218
max_diff_between_entry_and_expire : 132.77
exit of each contract : buy_call[0.1], sell_call[0.1], buy_put[232.0], sell_put[32.5]
pnl : 82.5
==================== expire report of 2024-01-11====================
middle line : 17500
used strike : {'buy_call': 17600, 'buy_put': 17400, 'sell_call': 17800, 'sell_put': 17200}
days to expire : 6
entry of each contract : buy_call[113.0], sell_call[49.0], buy_put[115.0], sell_put[59.0]
entry cost : -120.0
minimum diff by entry : 221
max_diff_between_entry_and_expire : 132.77
exit of each contract : buy_call[0.1], sell_call[0.1], buy_put[232.0], sell_put[32.5]
pnl : 79.5
==================== expire report of 2024-01-

In [173]:
daytrade_cnt = 0
expire_trade_cnt = 0
daytrade_pnl = 0 
expirce_pnl = 0 
for k, v in middles.items():
    if 'daytrade_pnl' in v:
        if v['daytrade_pnl'] is not None:
            daytrade_pnl += v['daytrade_pnl']
            daytrade_cnt += 1
    if 'expire_pnl' in v:
        if v['expire_pnl'] is not None:
            expirce_pnl += v['expire_pnl']
            expire_trade_cnt += 1

In [175]:
daytrade_cnt, expire_trade_cnt, daytrade_pnl, expirce_pnl, round(daytrade_pnl / daytrade_cnt, 2), round(expirce_pnl / expire_trade_cnt , 2)

(72, 72, -74.29999999999998, 574.9, -1.03, 7.98)